# Get daily Wikipedia pageviews for topic(s)

#### Import Python tools

In [3]:
%load_ext lab_black

In [4]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np

In [5]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [6]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [7]:
today = pd.to_datetime("today").strftime("%Y%m%d00")

---

## Enter two topics to compare

In [8]:
topics = ["Wednesday_Addams", "Cat"]
# topics = ["Recession"]

In [9]:
dataframes = []

for t in topics:
    df_src = pd.json_normalize(
        pd.read_json(
            f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/user/{t}/daily/2015070100/{today}"
        )["items"]
    ).sort_values("views", ascending=False)
    dataframes.append(df_src)
    src = pd.concat(dataframes)

In [10]:
src["date"] = pd.to_datetime(src["timestamp"].str[:8])

In [11]:
src["subject"] = src["article"].str.replace("_", " ", regex=False)

In [12]:
df = src.drop(["article", "project", "timestamp", "access", "agent"], axis=1)

In [13]:
df.head()

,granularity,views,date,subject
2703,daily,54811,2022-11-24,Wednesday Addams
2702,daily,27175,2022-11-23,Wednesday Addams
2605,daily,22580,2022-08-18,Wednesday Addams
122,daily,20986,2015-10-31,Wednesday Addams
1949,daily,19414,2020-10-31,Wednesday Addams


In [14]:
df.tail()

,granularity,views,date,subject
1620,daily,5277,2019-12-07,Cat
1431,daily,5239,2019-06-01,Cat
907,daily,5215,2017-12-24,Cat
1627,daily,5133,2019-12-14,Cat
906,daily,5102,2017-12-23,Cat


---

In [15]:
alt.Chart(df).mark_bar(size=2).encode(
    x=alt.X("date", axis=alt.Axis(format="%b. %Y"), title=""),
    y=alt.Y("views", title="", axis=alt.Axis(tickCount=4)),
    facet=alt.Facet("subject", columns=2, title=" "),
).configure_legend(orient="top").properties(
    width=400,
    height=300,
    title=f"Pageviews on Wikipedia",
)

/Users/mstiles/.local/share/virtualenvs/notebooks-pUaZ2fM9/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [16]:
df.groupby("subject").agg({"views": sum}).reset_index()

,subject,views
0,Cat,32089202
1,Wednesday Addams,3659027
